Import

In [1]:
from collections import Counter
import pandas as pd
import json
import os
import re

In [2]:
import argparse

In [3]:
import timeit
import time
%load_ext Cython
import gc

read params from commandline input:
- input folder read by wiki-extraktor
- output folder(s?): csv(for matrix), intermediate steps(single dicts for each read-in file, complete dict for complete wordcounts (all in JSON?)

(edit section below, only example)

In [4]:
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--inputpath", required=True, help="path to input dataset")
ap.add_argument("-o", "--outputpath", required=True, help="path to output model")
#ap.add_argument("-p", "--plot", type=str, default="plot.png", help="path to output accuracy/loss plot")
args = vars(ap.parse_args())

usage: ipykernel_launcher.py [-h] -d DATASET -m MODEL
ipykernel_launcher.py: error: the following arguments are required: -d/--dataset, -m/--model


SystemExit: 2

/home/hautakivi/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


read stopwordlist, which folder?

In [5]:
with open('stopwords.txt', 'r') as s:
    e = s.readlines()
stopwords = [x.strip() for x in e]

folder to saved wiki-extractor-files

In [6]:
folder = '/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/'
#later something like:
#folder = sorted(list(paths.list_images(args["dataset"])))
#folder = sorted(list(args["dataset"]))

create list of files of wiki-extractor-files to iterate over:

In [12]:
def processFolder(folder):
    pathlist = []
    for subdir, dirs, files in os.walk(folder):
        for file in files:
            pathlist.append(os.path.join(subdir, file))
            print(subdir,'\t', file )

    return pathlist

pathlist = processFolder(folder)

/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BT 	 wiki_99
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BT 	 wiki_93
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BT 	 wiki_98
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BT 	 wiki_21
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BT 	 wiki_72
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BT 	 wiki_77
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BT 	 wiki_18
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BT 	 wiki_23
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BT 	 wiki_57
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BT 	 wiki_24
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BT 	 wiki_80
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BT 	 wiki_94
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BT 	 wiki_67
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BT 	 wiki_65
/home/hautakivi/PyhtonProjects/wik

/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AL 	 wiki_64
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AL 	 wiki_87
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AL 	 wiki_33
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AL 	 wiki_09
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AL 	 wiki_30
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AL 	 wiki_60
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AL 	 wiki_55
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AL 	 wiki_10
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AL 	 wiki_50
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AL 	 wiki_58
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AL 	 wiki_63
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AL 	 wiki_81
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AL 	 wiki_36
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AL 	 wiki_75
/home/hautakivi/PyhtonProjects/wik

/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BM 	 wiki_99
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BM 	 wiki_93
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BM 	 wiki_98
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BM 	 wiki_21
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BM 	 wiki_72
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BM 	 wiki_77
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BM 	 wiki_18
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BM 	 wiki_23
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BM 	 wiki_57
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BM 	 wiki_24
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BM 	 wiki_80
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BM 	 wiki_94
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BM 	 wiki_67
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BM 	 wiki_65
/home/hautakivi/PyhtonProjects/wik

/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BE 	 wiki_57
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BE 	 wiki_24
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BE 	 wiki_80
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BE 	 wiki_94
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BE 	 wiki_67
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BE 	 wiki_65
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BE 	 wiki_22
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BE 	 wiki_19
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BE 	 wiki_31
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BE 	 wiki_88
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BE 	 wiki_92
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BE 	 wiki_07
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BE 	 wiki_12
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BE 	 wiki_56
/home/hautakivi/PyhtonProjects/wik

/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BZ 	 wiki_57
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BZ 	 wiki_24
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BZ 	 wiki_80
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BZ 	 wiki_94
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BZ 	 wiki_67
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BZ 	 wiki_65
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BZ 	 wiki_22
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BZ 	 wiki_19
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BZ 	 wiki_31
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BZ 	 wiki_88
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BZ 	 wiki_92
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BZ 	 wiki_07
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BZ 	 wiki_12
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BZ 	 wiki_56
/home/hautakivi/PyhtonProjects/wik

/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/CB 	 wiki_21
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/CB 	 wiki_72
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/CB 	 wiki_77
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/CB 	 wiki_18
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/CB 	 wiki_23
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/CB 	 wiki_57
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/CB 	 wiki_24
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/CB 	 wiki_80
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/CB 	 wiki_67
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/CB 	 wiki_65
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/CB 	 wiki_22
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/CB 	 wiki_19
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/CB 	 wiki_31
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/CB 	 wiki_07
/home/hautakivi/PyhtonProjects/wik

/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AM 	 wiki_97
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AM 	 wiki_48
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AM 	 wiki_00
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AM 	 wiki_35
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AM 	 wiki_91
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AM 	 wiki_82
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AM 	 wiki_39
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AM 	 wiki_40
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AM 	 wiki_28
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AM 	 wiki_41
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AM 	 wiki_69
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AM 	 wiki_78
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AM 	 wiki_62
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AM 	 wiki_51
/home/hautakivi/PyhtonProjects/wik

/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AC 	 wiki_34
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AC 	 wiki_11
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AC 	 wiki_73
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AC 	 wiki_46
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AC 	 wiki_26
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AC 	 wiki_32
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AC 	 wiki_17
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AC 	 wiki_96
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AC 	 wiki_42
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AC 	 wiki_15
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AC 	 wiki_61
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AC 	 wiki_37
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AC 	 wiki_68
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/AC 	 wiki_95
/home/hautakivi/PyhtonProjects/wik

functions to

1) check, if files are already exist in intermediate-outputs

2) read extractor-files, lower, split, output-per-extractor-files

(some duplicate imports in case that cython is used, so in case of stopwordlist creation)

(ToDo: add function to split corpus into sentences)

In [28]:
'''
%%cython
import json
import time
from collections import Counter
import re
import gc
'''

# stopwordlist in case of cython usage
with open('stopwords.txt', 'r') as s:
    e = s.readlines()
stopwords = [x.strip() for x in e]

# import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

# check, if files already handled (not functioning properly)
def fileChecker(path):
    s = re.findall(r'\w+|\w+$',path)
    s = '_'.join([s[-2], s[-1]])
    done = os.path.isfile('/home/hautakivi/PyhtonProjects/coocurrence/CounterDicts/{}'.format(s))
    if not done:
        print(s, done)
        print(path)
        return path
    
# read wiki-extractor (have to be done with --json flag, easier to serialize, read and parse)
# first: tokenize to words
def readJson(path):
    # for case of using function in loop: empty created lists/dicts/Counters (for memory-saving(?))
    sentencelist = []
    wordlists = []
    
    data = []
    wordlist = []
    wordlists = []
    c = {}

    # read and lower text in files
    with open(path, 'r') as f:
        
        data = [json.loads(x)['text'] for x in f]
        
        #wordlists = [x.lower().split() for x in data]
        #last = time.time()
        
        sentencelist = [sentencelist + sent_tokenize(x.lower()) for x in data]        
        
        #wordlist = [word_tokenize(y) for y in x for x in sentencelist]
        for x in sentencelist:
            for y in x:
                wordlists.append(word_tokenize(y))
        
        # was below:
        #wordlists = [word_tokenize(x.lower()) for x in data]
        
        
        #sent_list = [sent_tokenize(x.lower()) for x in data]
        #print(time.time() - last)
        
        # remove stopwords (with set())
        #wordlist = []
        for words in wordlists:
            #doesn't apply the set on the words, so keeps original number of words(?!)
            wordlist += set(words).difference(stopwords)
        
        
        # create Counter
        #wordlist = [(set(x).difference(stopwords)) for x in wordlists]
        c = Counter(wordlist)
        
        # create fitting outputfile-names
        s = re.findall(r'\w+|\w+$',path)
        #s = '_'.join([s[-2], s[-1]])
        
        # write single dict(JSON)-files for better handling (smaller files)
        # ToDo: write files in subfolder, not all in one directory!
        
        folder = '/home/hautakivi/PyhtonProjects/coocurrence/CounterDicts/'
        subfolder = os.path.join(folder, s[-2])
        
        if not os.path.exists(subfolder):
            os.makedirs(subfolder)
        
        #with open('/home/hautakivi/PyhtonProjects/coocurrence/CounterDicts/{}'.format(s), 'w', encoding='utf-8') as w:
        with open('{}/{}'.format(subfolder, s[-1]), 'w', encoding='utf-8') as w:
            json.dump(c, w, ensure_ascii=False)
        #print(s)
        #collected = gc.collect()
        #print('Garbage collector collected {}'.format(collected))
    return (sentencelist, c)

def readTokenizeSentence(path):
    sentencelist = []
    
    with open(path, 'r') as f:
        data = [json.loads(x)['text'] for x in f]
        
        sentencelist = [sent_tokenize(x.lower()) for x in data]
        #print(sentencelist)
        
        return sentencelist

(add own kwic-function from other notebook here)

(note: "documents" are sentences (sent_tokenize...))

In [29]:
# for developing purposes: process only a number of files and time it
start = time.time()
c = Counter()
liste = []
for i in range(5):
    last = time.time()
    sentList, c = readJson(pathlist[i])
    c += c
    #liste.append(readJson(pathlist[i]))
    
    #liste.append(readTokenizeSentence(pathlist[i]))
    #print(liste)
    print(i, round(time.time() - last, 2))

BT 	 wiki_99
0 2.39
BT 	 wiki_93
1 2.37
BT 	 wiki_98
2 2.26
BT 	 wiki_21
3 2.27
BT 	 wiki_72
4 2.29


In [62]:
toDoPaths = [fileChecker(x) for x in pathlist]
len(toDoPaths)

BP_wiki_17 False
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BP/wiki_17
BP_wiki_96 False
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BP/wiki_96
BP_wiki_42 False
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BP/wiki_42
BP_wiki_15 False
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BP/wiki_15
BP_wiki_61 False
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BP/wiki_61
BP_wiki_37 False
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BP/wiki_37
BP_wiki_68 False
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BP/wiki_68
BP_wiki_95 False
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BP/wiki_95
BP_wiki_13 False
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BP/wiki_13
BP_wiki_25 False
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BP/wiki_25
BP_wiki_89 False
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BP/wiki_89
BP_wiki_71 False
/home/hautakivi/PyhtonProjects/wikiextractor/extracted2/BP/

5381

In [63]:
toDoPaths

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

read all files and put out Counter-files (to be summed up together later)

In [8]:
start = time.time()
c = Counter()
for n, x in enumerate(pathlist):
    print('{}%'.format(round(n/len(pathlist)*100, 2)))
    c += readJson(x)
end = time.time()
print (end-start)

0.0%
Garbage collector collected 11229
0.02%
Garbage collector collected 33
0.04%


KeyboardInterrupt: 

do the same using multiprocessing: prarallelizing tasks, to be quicker (tried with 8 processes)

In [ ]:
# ToDo: Memory still running full, aborting at about 5000 of 5400 files...
import multiprocessing as mp

start = time.time()
pool = mp.Pool(processes=6)
#results = [pool.apply(cube, args=(x,)) for x in range(1,7)]
#results = pool.map(cube, range(8))
results = pool.map(readJson, pathlist)
print(results)
print('time elapsed: {}', time.time())

Garbage collector collected 10
Garbage collector collected 15
Garbage collector collected 20
Garbage collector collected 10
Garbage collector collected 10
Garbage collector collected 10
Garbage collector collected 10
Garbage collector collected 10
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Ga

Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage co

Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage co

Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage co

Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage co

Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage collector collected 0
Garbage co

read in created CounterDicts to sum single files up

(Memory-usage nearly not increased during process! --> seems efficient)

In [7]:
folderRead = '/home/hautakivi/PyhtonProjects/coocurrence/CounterDicts/'
readIn = processFolder(folderRead)

In [21]:
dic = Counter()
i = 0
for each in readIn:
    i += 1
    with open(each, 'r') as w:
        dic = Counter(dic + Counter(json.load(w)))
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [24]:
with open('/home/hautakivi/PyhtonProjects/coocurrence/tokenized_nltk', 'w') as j:
    json.dump(dic, j)

In [22]:
dic

Counter({'okurowen': 1,
         'apartheid-kritikern': 1,
         'feinmechanikerfamilie': 2,
         'eiff': 9,
         'reiter-brüdern': 1,
         'textilkleiderfabrik': 1,
         'mimicha': 1,
         'offen-werden': 1,
         'about.tml': 1,
         '1529–1572': 1,
         'synonymen': 255,
         'metallah': 1,
         'wiederdurchblutung': 1,
         'seitenlang': 9,
         'usb-organisation': 1,
         'pflichten.': 2,
         'wonei': 2,
         'himmelshund': 3,
         'ancona–lecce': 6,
         'speisefettherstellung': 1,
         'angriffstaste': 1,
         'portaria': 4,
         'estadistico': 1,
         'verträten': 25,
         'voreifeler': 1,
         'vallisnieri': 2,
         'zuchtleistungskontrollen': 1,
         '1644–1717': 6,
         'bibliotheksforschung': 1,
         'südkometen': 4,
         'frachtlinie': 14,
         'colangelo': 8,
         'liutprandina': 1,
         'israyel': 1,
         'parabutlerit': 1,
         'minoisch

In [28]:
pd.DataFrame.from_dict(dic, orient='index').sort_values(0, ascending=False)[:10000]

,0
.,953162
",",908331
),857626
(,857615
'',600405
``,599512
:,458835
*,346284
;,325280
“,303001
